# DSCI 100/Section 009/Group 169 Group Project Proposal

### [data_set_name] Analysis Proposal
By: Shady Abo El Kasim, Nalan Goosen, Labella Li, Yusen Wu

#### Introduction


* *Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal*
* *Clearly state the question you will try to answer with your project*
    * predictive question: asks about predictions of measurements or labels, what and not why (ex. "what will an individual choose based on prior data?")
* *Identify and describe the dataset that will be used to answer the question*
    * identify origins of dataset/brief description of organization, when the dataset was pulled

In [1]:
### Please run this cell before continuing.

library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
set.seed(2022) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

#### Preliminary Exploratory Data Analysis

* *Demonstrate that the dataset can be read from the web into R*
* *Clean and wrangle your data into a tidy format*
* *Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data.*
    * initial_split 
    * group_by, summarize
    * averages
    * map_df?
    * ^ allowed to use more than one table for several functions
* *Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.*
    * discuss which variables to use in analysis

In [2]:
#reading and cleaning dataset

heart_disease_data <- read_csv("https://raw.githubusercontent.com/labellali/dsci-100-2022w1-group-169/main/data/heart_disease_dataset.csv")
heart_disease_data <- heart_disease_data |>
    rename(age = Column1,
            sex = Column2,
                            chest_pain = Column3,
                            resting_blood_pressure = Column4,
                            cholesterol = Column5,
                            fasting_blood_sugar = Column6,
                            rest_ecg = Column7,
                            max_heart_rate = Column8,
                            exercised_ind_angina = Column9,
                            oldpeak = Column10,
                            slope = Column11,
                            ca = Column12,
                            thal = Column13,
                            num = Column14) |>
    mutate(sex = as.factor(sex),
          chest_pain = as.factor(chest_pain),
          fasting_blood_sugar = as.factor(fasting_blood_sugar),
          rest_ecg = as.factor(rest_ecg),
          exercised_ind_angina = as.factor(exercised_ind_angina),
          num = as.factor(num),
          thal = na_if(thal, '?'),
          ca = na_if(ca, '?'),
          thal = as.numeric(thal),
          ca = as.numeric(ca))

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): Column12, Column13
dbl (12): Column1, Column2, Column3, Column4, Column5, Column6, Column7, Col...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# splitting data into initial and training datasets

heart_disease_split <- initial_split(heart_disease_data, prop = 0.75, strata = num)
training_data <- training(heart_disease_split)
testing_data <- testing(heart_disease_split)

[brief explanation of exploratory data analysis choices]

In [4]:
# exploratory data analysis

# looking at the number of observations in each class to predict

training_data_count <- training_data |>
    group_by(num) |>
    summarize(n = n())

In [5]:
# calculating the means of every numeric predictor

training_data_means <- training_data |>
    select(age, resting_blood_pressure, cholesterol, max_heart_rate, oldpeak, slope, thal, ca) |>
    map_df(mean, na.rm = TRUE)

In [16]:
# counting how many rows have missing data

training_data_na_count <- training_data |>
    filter(age == is.na(age) ||
          sex == is.na(sex) ||
          chest_pain == is.na(chest_pain) ||
          fasting_blood_sugar == is.na(fasting_blood_sugar) ||
          rest_ecg == is.na(rest_ecg) ||
          max_heart_rate == is.na(max_heart_rate) ||
          exercised_ind_angina == is.na(exercised_ind_angina) ||
          oldpeak == is.na(oldpeak) ||
          slope == is.na(slope) ||
          ca == is.na(ca) ||
          thal == is.na(thal) ||
          num == is.na(num)) |>
    summarize(rows_with_na = n())
training_data_na_count

rows_with_na
<int>
225


#### Method

* *Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?*
    * filter for certain country/time range
    * select(-removal) any unnecessary columns
* *Describe at least one way that you will visualize the results*
    * consider facet_grid?
    * different kinds of geom visualization functions

- **note**: while cleaning the dataset and looking at the variables, i thought that ca, oldpeak, and slope were probably removable variables if end up going with this dataset

#### Expected outcomes and significance

* *What do you expect to find?*
* *What impact could such findings have?*
* *What future questions could this lead to?*